In [1]:
import pandas as pd
import json
from pathlib import Path

dir_gto_zora = Path.cwd().joinpath('calcs_gto_zora')
dir_mw_zora = Path.cwd().joinpath('calcs_mw_zora')
dir_gto_nrel_ugbs = Path.cwd().joinpath('calcs_gto_nrel_ugbs')
dir_gto_nrel_sarc = Path.cwd().joinpath('calcs_gto_nrel_sarczorabasis')
dir_mw_nrel = Path.cwd().joinpath('calcs_mw_nrel')

In [2]:
def getEnergy(f: Path, code: str) -> float:
    """"""
    if code in ['orca', 'o']:
        with f.open() as infile:
            for line in infile.readlines():
                if line.strip().startswith('FINAL SINGLE POINT ENERGY'):
                    return float(line.split()[4])
    elif code in ['mrchem', 'm']:
        with f.open() as infile:
            output = json.loads(infile.read())
            return output['output']['properties']['scf_energy']['E_tot']

## ZORA

In [3]:
table_gto = []
table_mw = []
mols = []

for f in sorted(dir_gto_zora.glob('*.out')):
    mol = f.stem
    mols.append(mol)
    e = getEnergy(f, 'o')
    table_gto.append((e))

for f in sorted(dir_mw_zora.glob('*.json')):
    if '_init' in str(f):
        continue
    mol = f.stem
    e = getEnergy(f, 'm')
    table_mw.append((e))
    
cols = ['Molecule', 'SARCZORA', 'MW6']
table = [mols, table_gto, table_mw]
df = pd.DataFrame(table).transpose()
df.columns = cols

df["Error"] = df.SARCZORA - df.MW6
print(df.to_latex(index=False))

\begin{tabular}{llll}
\toprule
Molecule &      SARCZORA &           MW6 &       Error \\
\midrule
     AgF &  -5296.642029 &  -5505.758184 &  209.116155 \\
     AgH &  -5197.401414 &   -5406.00292 &  208.601506 \\
     AgO &  -5271.925482 &  -5480.902526 &  208.977044 \\
      I2 & -13829.724739 & -14536.143654 &  706.418915 \\
     SrF &  -3231.984865 &  -3316.080062 &   84.095198 \\
     SrH &  -3132.661175 &  -3216.372295 &    83.71112 \\
     SrO &  -3207.299624 &  -3291.305737 &   84.006114 \\
\bottomrule
\end{tabular}



## NREL

In [4]:
table_ugbs = []
table_sarc = []
table_mw = []
mols = []

for f in sorted(dir_gto_nrel_ugbs.glob('*.out')):
    mol = f.stem
    mols.append(mol)
    e = getEnergy(f, 'o')
    table_ugbs.append((e))
    
for f in sorted(dir_gto_nrel_sarc.glob('*.out')):
    mol = f.stem
    e = getEnergy(f, 'o')
    table_sarc.append((e))

for f in sorted(dir_mw_nrel.glob('*.json')):
    if '_init' in str(f):
        continue
    mol = f.stem
    e = getEnergy(f, 'm')
    table_mw.append((e))
    
cols = ['Molecule', 'UGBS', 'SARC', 'MW6']
table = [mols, table_ugbs, table_sarc, table_mw]

df = pd.DataFrame(table).transpose()
df.columns = cols
print(df.to_latex())

\begin{tabular}{lllll}
\toprule
{} & Molecule &          UGBS &          SARC &           MW6 \\
\midrule
0 &      AgF &   -5299.38894 &  -5296.642029 &  -5299.398107 \\
1 &      AgH &  -5200.152722 &  -5197.401414 &   -5200.15549 \\
2 &      AgO &  -5274.669595 &  -5271.925482 &  -5274.650578 \\
3 &       I2 & -13840.147255 & -13829.724739 & -13840.157836 \\
4 &      SrF &  -3232.834304 &  -3231.984865 &  -3232.846756 \\
5 &      SrH &  -3133.514795 &  -3132.661175 &  -3133.520968 \\
6 &      SrO &  -3208.140034 &  -3207.299624 &  -3208.161674 \\
\bottomrule
\end{tabular}



In [5]:
df["Error-UGBS"] = df.UGBS - df.MW6
df["Error-SARC"] = df.SARC - df.MW6
print(df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
Molecule &          UGBS &          SARC &           MW6 & Error-UGBS & Error-SARC \\
\midrule
     AgF &   -5299.38894 &  -5296.642029 &  -5299.398107 &   0.009168 &   2.756078 \\
     AgH &  -5200.152722 &  -5197.401414 &   -5200.15549 &   0.002768 &   2.754076 \\
     AgO &  -5274.669595 &  -5271.925482 &  -5274.650578 &  -0.019017 &   2.725096 \\
      I2 & -13840.147255 & -13829.724739 & -13840.157836 &   0.010581 &  10.433097 \\
     SrF &  -3232.834304 &  -3231.984865 &  -3232.846756 &   0.012452 &   0.861892 \\
     SrH &  -3133.514795 &  -3132.661175 &  -3133.520968 &   0.006173 &   0.859793 \\
     SrO &  -3208.140034 &  -3207.299624 &  -3208.161674 &    0.02164 &    0.86205 \\
\bottomrule
\end{tabular}



In [6]:
pd.DataFrame.to_latex?

Signature:
pd.DataFrame.to_latex(
    self,
    buf=None,
    columns=None,
    col_space=None,
    header=True,
    index=True,
    na_rep='NaN',
    formatters=None,
    float_format=None,
    sparsify=None,
    index_names=True,
    bold_rows=False,
    column_format=None,
    longtable=None,
    escape=None,
    encoding=None,
    decimal='.',
    multicolumn=None,
    multicolumn_format=None,
    multirow=None,
    caption=None,
    label=None,
    position=None,
)
Docstring:
Render object to a LaTeX tabular, longtable, or nested table/tabular.

Requires ``\usepackage{booktabs}``.  The output can be copy/pasted
into a main LaTeX document or read from an external file
with ``\input{table.tex}``.

.. versionchanged:: 1.0.0
   Added caption and label arguments.

.. versionchanged:: 1.2.0
   Added position argument, changed meaning of caption argument.

Parameters
----------
buf : str, Path or StringIO-like, optional, default None
    Buffer to write to. If None, the output is returne